In [44]:
import numpy as np
import matplotlib.pyplot as plt
import copy
import math
import re
import linecache

## Bond length and angles

In [46]:
num_mole=1000   # define the number of molecules

list1=[[] for i in range(3*num_mole)]   # store the coordinates of 3000 atoms
i=0
with open("/Users/stephen/desktop/2nd_python/lammps_1000_xyz.txt", "r") as f:  # read the file created by VMD
    for line in f.readlines():
        line = line.strip('\n')
        par='-?\d+\.*\d*'
        line_num = re.compile(par).findall(line)    # use Regular Expression to find all the numbers and numerical values in a string(integer, decimal,negative number)
        for c in line_num:
            list1[i].append(float(c))   # convert to float
        i+=1
        
for j in range(3*num_mole):
    print(list1[j])

[1.0, 1.0, 2.0, -1.1128, 4.539, 8.546, 27.695999]
[2.0, 1.0, 1.0, 0.5564, 3.605, 8.417, 27.530001]
[5.0, 2.0, 1.0, 0.5564, 20.476, 25.780001, 26.757]
[4.0, 2.0, 2.0, -1.1128, 21.313999, 26.238001, 26.811001]
[3.0, 1.0, 1.0, 0.5564, 4.682, 9.481, 27.547001]
[6.0, 2.0, 1.0, 0.5564, 21.127001, 27.02, 27.330999]
[7.0, 3.0, 2.0, -1.1128, 0.621, 7.761, 2.282]
[8.0, 3.0, 1.0, 0.5564, 0.413, 6.833, 2.173]
[9.0, 3.0, 1.0, 0.5564, 0.002, 8.215, 1.709]
[10.0, 4.0, 2.0, -1.1128, 24.496, 15.72, 2.32]
[11.0, 4.0, 1.0, 0.5564, 23.747999, 16.021999, 1.805]
[12.0, 4.0, 1.0, 0.5564, 24.516001, 14.773, 2.182]
[13.0, 5.0, 2.0, -1.1128, 28.603001, 8.908, 26.542]
[14.0, 5.0, 1.0, 0.5564, 28.459, 7.965, 26.469]
[15.0, 5.0, 1.0, 0.5564, 29.392, 8.989, 27.079]
[16.0, 6.0, 2.0, -1.1128, 2.869, 4.479, 5.915]
[17.0, 6.0, 1.0, 0.5564, 3.092, 3.901, 6.644]
[18.0, 6.0, 1.0, 0.5564, 2.545, 3.891, 5.233]
[19.0, 7.0, 2.0, -1.1128, 3.625, 12.796, 16.722]
[20.0, 7.0, 1.0, 0.5564, 4.56, 12.595, 16.740999]
[21.0, 7.0, 1.0,

#### Define the subfunctions to calculate the H-O bond length and H-O-H angle.

In [47]:
def length(a1,b1,c1,a2,b2,c2):
    d=np.sqrt((a1-a2)**2+(b1-b2)**2+(c1-c2)**2)        
    return d

def angle(d1,d2,d3):
    a=math.acos((d1**2+d2**2-d3**2)/(2*d1*d2))*180/math.pi  # convert radians to degree
    return a

#### Find the average H-O bond length, average H-O-H angle, largest deviation of bond length and angle.

In [48]:

sum1=0
sum2=0
sum3=0

for i in range(num_mole):
    sum1=sum1+list_out[i][0]
    sum2=sum2+list_out[i][1]
    sum3=sum3+list_out[i][2]
    
avg1=sum1/num_mole
avg2=sum2/num_mole
avg12=(avg1+avg2)/2
avg3=sum3/num_mole


list_dev=[[] for i in range(num_mole)]
for i in range(num_mole):
    for j in range(2):
        list_dev[i].append(list_out[i][j]-avg12)

#print(list_dev)
max1=0
ilist=[]
jlist=[]
for i in range(num_mole):
    for j in range(2):
        if list_dev[i][j]>max1:
            max1=list_dev[i][j]
            #print("i list",i+1)
            ilist.append(i+1)
            #print("j list",j+1)
            jlist.append(j+1)

list_dev2=[]    
for i in range(num_mole):
    list_dev2.append(list_out[i][2]-avg3)

alist=[]
max2=0
for a in range(num_mole):
    if list_dev2[a]>max2:
        max2=list_dev2[a]
        #print("a list",a+1)
        alist.append(a+1)


#### Output the H-O bond length and H-O-H angle of each water molecule, and detect the wrong order and connections.

In [49]:
list_out=[[] for i in range(num_mole)]
i=0
m=0
while i <=3*num_mole-2:
    j=i+1
    k=i+2
    dd1=length(list1[i][4], list1[i][5], list1[i][6], list1[j][4], list1[j][5], list1[j][6])
    dd2=length(list1[i][4], list1[i][5], list1[i][6], list1[k][4], list1[k][5], list1[k][6])
    dd3=length(list1[j][4], list1[j][5], list1[j][6], list1[k][4], list1[k][5], list1[k][6])
    list_out[m].append(dd1)
    list_out[m].append(dd2)
    list_out[m].append(angle(dd1,dd2,dd3))
    i=i+3
    m=m+1


i1=0
m=0
a=1
while i1 <=3*num_mole-3:
    j1=i1+1
    k1=i1+2
    if list1[i1][0]!=list1[j1][0]-1:
        print("First hydrogen out of order in molecule", a)
        m+=1
    if list1[i1][0]!=list1[k1][0]-2:
        print("Second hydrogen out of order in molecule", a)
        m+=1
    i1=i1+3
    a=a+1

if m==0:
    print("All the atoms are in correct order") 

    
    
print("The average bond length is : ", avg12)
print("The average angle is : ", avg3)
print("The largest distance deviation is ", max1, "in the", jlist[-1],"th bond of molecule", ilist[-1])
print("The largest angle deviation is ", max2, "in molecule", alist[-1])
print("\n")

print("Molecule No.        First bond length        Second bond length        Angle")

for j in range(3*num_mole):
    print("Molecule", j+1,":  ", list_out[j])        

Second hydrogen out of order in molecule 1
First hydrogen out of order in molecule 2
The average bond length is :  0.9573078494529116
The average angle is :  104.51232578247254
The largest distance deviation is  0.0011539390394624816 in the 1 th bond of molecule 719
The largest angle deviation is  0.1269908259979502 in molecule 440


Molecule No.        First bond length        Second bond length        Angle
Molecule 1 :   [0.9573673986532025, 23.492136526718934, 138.9169577940285]
Molecule 2 :   [23.621222856787114, 0.9575430058274121, 115.14589445258711]
Molecule 3 :   [0.957250750848491, 0.9579175329849641, 104.53309388809278]
Molecule 4 :   [0.9570443521603363, 0.9572110739021994, 104.53310022409596]
Molecule 5 :   [0.9567205903507038, 0.9578358011689686, 104.48431435179343]
Molecule 6 :   [0.9566890821996457, 0.9569973876662364, 104.52138705643034]
Molecule 7 :   [0.9565495083899205, 0.9580864887895046, 104.50060014137881]
Molecule 8 :   [0.9571237088286975, 0.9571787095422669, 1

IndexError: list index out of range

#### Read the H-O bonds and detect the wrong connections

In [51]:
### use separated file: lammps_1000_bond.txt

list_con=[[] for i in range(2*num_mole)]
i=0
with open("/Users/stephen/desktop/2nd_python/lammps_1000_bond.txt", "r") as f:
    for line in f.readlines():
        line = line.strip('\n')
        par='-?\d+\.*\d*'
        line_num = re.compile(par).findall(line)
        for c in line_num:
            list_con[i].append(int(c))
        i+=1       
        
# check

i=0
ii=0
a=1
while i <= 2*num_mole-2:
    if list_con[i][3]-list_con[i][2]!=1: 
        print("Wrong connection in the first H-O bond of molecule",a)
        ii+=1
    if list_con[i+1][3]-list_con[i+1][2]!=2:
        print("Wrong connection in the second H-O bond of molecule",a)
        ii+=1    
    i=i+2
    a=a+1
    
if ii==0:
    print("All the connections are correct")
print("\n")

# output

j=0
a=0
while j <= 2*num_mole:
    print("Molecule", j+1-a,":  ",  list_con[j])
    print("Molecule", j+1-a,":  ",  list_con[j+1])
    j=j+2
    a=a+1

Wrong connection in the second H-O bond of molecule 1
Wrong connection in the first H-O bond of molecule 2


Molecule 1 :   [1, 1, 1, 2]
Molecule 1 :   [2, 1, 1, 5]
Molecule 2 :   [3, 1, 4, 3]
Molecule 2 :   [4, 1, 4, 6]
Molecule 3 :   [5, 1, 7, 8]
Molecule 3 :   [6, 1, 7, 9]
Molecule 4 :   [7, 1, 10, 11]
Molecule 4 :   [8, 1, 10, 12]
Molecule 5 :   [9, 1, 13, 14]
Molecule 5 :   [10, 1, 13, 15]
Molecule 6 :   [11, 1, 16, 17]
Molecule 6 :   [12, 1, 16, 18]
Molecule 7 :   [13, 1, 19, 20]
Molecule 7 :   [14, 1, 19, 21]
Molecule 8 :   [15, 1, 22, 23]
Molecule 8 :   [16, 1, 22, 24]
Molecule 9 :   [17, 1, 25, 26]
Molecule 9 :   [18, 1, 25, 27]
Molecule 10 :   [19, 1, 28, 29]
Molecule 10 :   [20, 1, 28, 30]
Molecule 11 :   [21, 1, 31, 32]
Molecule 11 :   [22, 1, 31, 33]
Molecule 12 :   [23, 1, 34, 35]
Molecule 12 :   [24, 1, 34, 36]
Molecule 13 :   [25, 1, 37, 38]
Molecule 13 :   [26, 1, 37, 39]
Molecule 14 :   [27, 1, 40, 41]
Molecule 14 :   [28, 1, 40, 42]
Molecule 15 :   [29, 1, 43, 44]
Mo

IndexError: list index out of range

#### Read the H-O-H angles and detect the wrong connections

In [52]:
### use separated file: lammps_1000_angle.txt

list_ang=[[] for i in range(num_mole)]
i=0
with open("/Users/stephen/desktop/2nd_python/lammps_1000_angle.txt", "r") as f:
    for line in f.readlines():
        line = line.strip('\n')
        par='-?\d+\.*\d*'
        line_num = re.compile(par).findall(line)
        for c in line_num:
            list_ang[i].append(int(c))
        i+=1

# check

i=0
ii=0
while i <= num_mole-1:
    if list_ang[i][4]!=list_ang[i][3]+2:
        print("Wrong angle! Molecule", i+1,":",list_ang[i][2],"--",list_ang[i][3],"--",list_ang[i][4])
        ii+=1
    if list_ang[i][2]!=list_ang[i][3]+1:
        print("Wrong angle! Molecule", i+1,":",list_ang[i][2],"--",list_ang[i][3],"--",list_ang[i][4])
        ii+=1    
    i=i+1
    
if ii==0:
    print("All the angles are correct")
print("\n")

# output

j=0
while j <= num_mole:
    print("Molecule", list_ang[j][0],":  ",  list_ang[j])
    j=j+1

All the angles are correct


Molecule 1 :   [1, 1, 2, 1, 3]
Molecule 2 :   [2, 1, 5, 4, 6]
Molecule 3 :   [3, 1, 8, 7, 9]
Molecule 4 :   [4, 1, 11, 10, 12]
Molecule 5 :   [5, 1, 14, 13, 15]
Molecule 6 :   [6, 1, 17, 16, 18]
Molecule 7 :   [7, 1, 20, 19, 21]
Molecule 8 :   [8, 1, 23, 22, 24]
Molecule 9 :   [9, 1, 26, 25, 27]
Molecule 10 :   [10, 1, 29, 28, 30]
Molecule 11 :   [11, 1, 32, 31, 33]
Molecule 12 :   [12, 1, 35, 34, 36]
Molecule 13 :   [13, 1, 38, 37, 39]
Molecule 14 :   [14, 1, 41, 40, 42]
Molecule 15 :   [15, 1, 44, 43, 45]
Molecule 16 :   [16, 1, 47, 46, 48]
Molecule 17 :   [17, 1, 50, 49, 51]
Molecule 18 :   [18, 1, 53, 52, 54]
Molecule 19 :   [19, 1, 56, 55, 57]
Molecule 20 :   [20, 1, 59, 58, 60]
Molecule 21 :   [21, 1, 62, 61, 63]
Molecule 22 :   [22, 1, 65, 64, 66]
Molecule 23 :   [23, 1, 68, 67, 69]
Molecule 24 :   [24, 1, 71, 70, 72]
Molecule 25 :   [25, 1, 74, 73, 75]
Molecule 26 :   [26, 1, 77, 76, 78]
Molecule 27 :   [27, 1, 80, 79, 81]
Molecule 28 :   [28, 1, 83

IndexError: list index out of range